# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import apikey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# File to Load (Remember to Change These)
csv_path = "../WeatherPy/output_data/cities.csv"

# Read Purchasing File and store into Pandas data frame
city_data_df = pd.read_csv(csv_path)
city_data_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Access maps with unique API key
gmaps.configure(api_key=apikey)

In [5]:
#Store Lat and Lng into coords
coords = city_data_df[['Lat','Lng']].astype(float)
humidity = city_data_df['Humidity'].astype(float)

#Create a heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(coords, weights=humidity)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Create new df
smallerweather_df = pd.DataFrame(city_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])
len(smallerweather_df)

#Drop null values (there are no null values)
cleanweather_df = smallerweather_df.dropna(axis=0, how='all')
len(cleanweather_df)

#Narrow down df to max temp >80, wind speed <10, and 0 clouidness
smallerweather_df = smallerweather_df[smallerweather_df['Max Temp'] >85]
smallerweather_df = smallerweather_df[smallerweather_df['Wind Speed'] < 10]
smallerweather_df = smallerweather_df[smallerweather_df['Cloudiness'] == 0]
smallerweather_df.head()

,City,Max Temp,Wind Speed,Cloudiness
26,colonelganj,94.20,5.08,0
142,sarai mir,86.00,3.36,0
244,pasni,86.82,9.17,0
416,kirkuk,86.82,7.36,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Create hotel_df
hotel_df = pd.DataFrame(city_data_df, columns=['City', 'Country', 'Lat', 'Lng'])
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,ostrovnoy,RU,68.05,39.51,
1,mahebourg,MU,-20.41,57.70,
2,qaanaaq,GL,77.48,-69.36,
3,zhuhai,CN,40.71,112.04,
4,cape town,ZA,-33.93,18.42,
...,...,...,...,...,...
543,yabrud,SY,33.97,36.66,
544,paraiso,MX,24.01,-104.61,
545,veraval,FR,49.65,0.71,
546,novyy urgal,RU,51.07,132.56,


In [22]:
# find the closest restaurant of each type to coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Create for loop to go through each row
for i, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    params = {
        "location": f"{lat},{lng}",
        "keyword": "hotel",
        "radius": 5000,
        "key": apikey
    }
    
    response = requests.get(base_url, params=params)
    places_data = response.json()
    
    print(f"Results for index {i}: {city}.")
    
    try:
        print(places_data["results"][0]['name'])
        hotel_df.loc[i, "Hotel Name"] = places_data["results"][0]['name']
        
    except (KeyError):
        print("Results not found.")
        
    print("----------")
print("-------Done--------")

Results for index 0: ostrovnoy.
Sberbank Rossii
----------
Results for index 1: mahebourg.
ONS MOTEL & GUEST HOUSE
----------
Results for index 2: qaanaaq.
Qaanaaq Hotel
----------
Results for index 3: zhuhai.


IndexError: list index out of range

In [ ]:
hotel_df

In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))